In [1]:
import sys
import requests
import csv
import json

In [2]:
def print_gap(num):
    for i in range(num): 
        print('')

In [3]:
def print_pattern():
    print('-' * 10)

In [6]:
def gen_api_url(filename):
    api_url = "https://api.github.com/repos/"
    with open(filename, "r") as file:
        file_data = json.load(file)      

    try:
        repo = file_data["repo_link"]
    except KeyError:
        print_gap(1)
        # print("Repo not stored.")
        repo = input("Enter Repo link: ")
        print_gap(1)

        # TODO: Add ability to store repo link
        # input("Do you want me to store repo? (Y/N): ")

    if len(repo.split('/')[-2:]) == 1:
        print('Invalid Url')
        return 'invalid'
    else:
        api_url += '/'.join(repo.split('/')[-2:])
        return api_url


In [7]:
## TODO: Add params. Only supports access_token now
def add_params_to_url(api_url, filename):
    with open(filename, "r") as file:
        file_data = json.load(file)   

    # try:
    #     access_token = file_data["access_token"]
    # except KeyError:
    #     print_pattern()
    #     # print("Access token not stored.")
    #     print("Access token required for private github repos, if public press enter(return), enter token otherwise.")
    #     print("Help on creating token - https://help.github.com/articles/creating-an-access-token-for-command-line-use/")
    #     print_pattern()
    #     print_gap(1)
    #     access_token = input("Enter Access Token: ")
    #     print_gap(1)

        # TODO: Add ability to store access_token
        # input("Do you want me to store access token? (Y/N): ")


    api_url += '/issues?state=closed'

    return api_url

Main

In [8]:
api_url = add_params_to_url(gen_api_url('data.txt'), 'data.txt')
print(api_url)
req = requests.get(api_url)
    # response = requests.get(api_url)
    # for issue in response.json():
        
    #     labels = issue['labels']
    #     for label in labels:
    #         print (label)
data = req.json()
    
output = open('output.txt', 'w+')
keys = data[0].keys()
output.write(" ".join(keys))



fields = ['number', 'title', 'html_url']

other = ['assignee', 'creater']

with open("data.csv", "w") as file:
    csv_file = csv.writer(file)
    cols = fields + other
    csv_file.writerow(cols)
    for item in data:
            row = [item[i] for i in fields]
            try:
                row += [item['assignee']['login']]
            except:


                row += [' ']
            row += [item['user']['login']]
            csv_file.writerow(row)

https://api.github.com/repos/Mahim1997/wQFM-2020/issues?state=closed
